<a href="https://colab.research.google.com/github/yj76461/DL_NLP/blob/main/py3_201710807.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#버전 확인
import sys
print(sys.version)

In [ ]:
#GPU사용여부
torch.cuda.is_available()

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
mnist_train=dset.MNIST("", train=True, transform=transforms.ToTensor(),target_transform=None,download=True)
mnist_test=dset.MNIST("", train=False, transform=transforms.ToTensor(),target_transform=None,download=True)

In [ ]:
print ("mnist_train 길이:", len(mnist_train))
print ("mnist_test 길이:", len(mnist_test))

image, label = mnist_train.__getitem__(0)
print ("image data 형태:", image.size())
print ("label: ", label)

img = image.numpy()
plt.title("label: %d" %label)
plt.imshow(img[0], cmap='gray')
plt.show()

In [ ]:
print(mnist_train[0][1])
print(mnist_train[0][0].size())

for i in range(3):
    img=mnist_train[i][0].numpy()
    print(mnist_train[i][1])
    plt.imshow(img[0],cmap='gray')
    plt.show()

In [ ]:
image, label = mnist_train[0]

image=image.view(-1, image.size()[0], image.size()[1], image.size()[2])
print(image.size())

print (label)

conv_layer=nn.Conv2d(in_channels=1,out_channels=3,kernel_size=3,padding=1)

output=conv_layer(Variable(image))
print(output.size())

for i in range(3):
    plt.imshow(output[0,i,:,:].data.numpy(), cmap='gray')
    plt.show()

In [ ]:
import numpy as np
import torch.optim as optim

batch_size = 16
learning_rate= 0.0002
num_epoch = 10

In [ ]:
train_loader = torch.utils.data.DataLoader(list(mnist_train)[:batch_size*100], batch_size=batch_size,
                                           shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader((mnist_test), batch_size=batch_size,
                                           shuffle=False, num_workers=2, drop_last=True)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer=nn.Sequential(
            nn.Conv2d(1,16,5,padding=2),
            nn.ReLU(),
            #nn.Dropout2d(0.2),
            #nn.BatchNorm2d(16),
            nn.Conv2d(16,32,5,padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            #nn.Dropout2d(0.2),
            #nn.BatchNorm2d(32),
            
            nn.Conv2d(32,64,5,padding=2),
            nn.ReLU(),
            
            #nn.Dropout2d(0.2),
            #nn.BatchNorm2d(64),

            nn.MaxPool2d(2,2)
        )
        self.fc_layer=nn.Sequential(
            nn.Linear(64*7*7, 100),
            nn.ReLU(),
            #nn.Dropout2d(0.2),
            #nn.BatchNorm1d(100),
            nn.Linear(100,10)
        )

        # for m in self.modules():
        #     if isinstance(m, nn.Conv2d):
        #         init.kaiming_normal(m.weight.data)
        #         m.bias.data.fill_(0)

        #     if isinstance(m, nn.Linear):
        #         init.kaiming_normal(m.weight.data)
        #         m.bias.data.fill_(0)

    def forward(self,x):
        out=self.layer(x)
        out=out.view(batch_size,-1)
        out=self.fc_layer(out)

        return out
model = CNN()

In [ ]:
for parameter in model.parameters():
    print(parameter)
    print(parameter.shape)

In [ ]:
loss_func=nn.CrossEntropyLoss()

#optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

#scheduler=lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.2)

#model.train()

for i in range(num_epoch):
    for j,[image, label] in enumerate(train_loader):
        x=Variable(image)
        y_=Variable(label)

        optimizer.zero_grad()
        output=model.forward(x)
        loss=loss_func(output,y_)
        loss.backward()
        optimizer.step()

        if j%50==0:
            print(loss, j, i)

In [ ]:
#save할때 ./drive/My Drive 부터 시작
torch.save(model, './drive/My Drive/Colab Notebooks/mycnn_model_%d.pkl'%(num_epoch))

In [ ]:
try:
    model=torch.load('./drive/My Drive/Colab Notebooks/mycnn_model_10.pkl')
    print("model restored")
except:
    print("model not restored")

In [ ]:
def ComputeAccr(dloader, imodel):
    correct = 0
    total = 0

    for j, [imgs, labels] in enumerate(dloader):
        img = Variable(imgs)
        label = Variable(labels)

        output = imodel.forward(img)
        _, output_index = torch.max(output, 1)

        total += label.size(0)
        correct += (output_index == label).sum().float()
    print("Accuracy of Test Data: {}".format(100*correct/total))
    

In [ ]:
ComputeAccr(test_loader,model)